In [64]:
import os
import numpy as np
import scipy.sparse.csgraph

In [65]:
INPUT_DIR = './inputs'
input0 = os.path.join(INPUT_DIR, '0.in')

In [66]:
def read_file(file):
    with open(file, 'r') as f:
        data = f.readlines()
    data = [line.strip().split() for line in data]
    return data

def data_parser(input_data):
    number_of_kingdoms = int(input_data[0][0])
    list_of_kingdom_names = input_data[1]
    starting_kingdom = input_data[2][0]
    adjacency_matrix = [[entry if entry == 'x' else float(entry) for entry in row] for row in input_data[3:]]
    return number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix

In [67]:
def matrix(input_file):
    adj = np.genfromtxt(input_file, dtype=float, skip_header=3)
    size = len(adj)
    for i in range(size):
        for j in range(size):
            if np.isnan(adj[i][j]):
                adj[i][j] = 0
    return adj

def preprocess(input_file):
    data = read_file(input_file)
    number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix = data_parser(data)
    adjacency_matrix = matrix(input_file)
    return number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix

number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix = preprocess(input0)

In [68]:
g = matrix(input0)
gnx =nx.from_numpy_matrix(g)
paths = list(nx.all_pairs_shortest_path(gnx))
recon = []
for p in paths:
    a = []
    for i in range(number_of_kingdoms):
        a.append(p[1][i])
    recon.append(a)
shortest_dist= scipy.sparse.csgraph.floyd_warshall(g)

In [69]:
complete_g = np.zeros_like(g)
for i in range(g.shape[0]):
    for j in range(g.shape[1]):
        if g[i,j] == np.inf:
            complete_g[i,j] = shortest_dist[i,j]
        else:
            complete_g[i,j] = g[i,j]
for kk in range(complete_g.shape[0]):
    complete_g[kk, kk]=0 
complete_g

array([[ 0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 2.,  3.,  0.,  3.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  2.,  0.,  1.,  0.,  1.,  0.,  2.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  2.,  0., 10.,  0., 10.,  0.,  2.,  0.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  0.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 

In [72]:
#Set Cover
binarize = lambda x : int(x != 0)
binarize = np.vectorize(binarize)
binary = binarize(g)
for i in range(number_of_kingdoms):
    binary[i][i] = 1
binary

array([[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]])

In [73]:
cost = np.diag(g)
cost = cost/ np.median(cost)

In [74]:
from SetCoverPy import *
sc = setcover.SetCover(binary, cost)
solution, time_used = sc.SolveSCP()
sc.s

This Best solution: UB=14.5, LB=14.499999999069434, UB1=14.5, LB1=14.499999999069434
Current Best Solution: UB=14.5, LB=14.499999999069434, change=6.4176954938061656e-09% @ niters=0
Current Best Solution: UB=14.5, LB=14.499999999069434, change=6.4176954938061656e-09% @ niters=1
Final Best solution: 14.5
Took 0.004 minutes to reach current solution.


array([ True,  True, False,  True, False,  True, False,  True, False,
        True,  True, False,  True,  True])

In [29]:
# from ortools.constraint_solver import pywrapcp
# from ortools.constraint_solver import routing_enums_pb2

# class CreateDistanceCallback(object):
#     """Create callback to calculate distances between points."""
#     def __init__(self, adjacency_matrix):
#         """Array of distances between points."""

#         self.matrix = adjacency_matrix

#     def Distance(self, from_node, to_node):
#         return int(self.matrix[from_node][to_node])

# def solve(list_of_kingdom_names, starting_kingdom, adjacency_matrix, params=[]):
# #     city_names = ["New York", "Los Angeles", "Chicago", "Minneapolis", "Denver", "Dallas", "Seattle",
# #                 "Boston", "San Francisco", "St. Louis", "Houston", "Phoenix", "Salt Lake City"]
#     tsp_size = len(list_of_kingdom_names)
#     num_routes = 1    # The number of routes, which is 1 in the TSP.
#     # Nodes are indexed from 0 to tsp_size - 1. The depot is the starting node of the route.
#     depot = list_of_kingdom_names.index(starting_kingdom)

#     # Create routing model
#     if tsp_size > 0:
#         routing = pywrapcp.RoutingModel(tsp_size, num_routes, depot)
#         search_parameters = pywrapcp.RoutingModel.DefaultModelParameters()

#         # Create the distance callback, which takes two arguments (the from and to node indices)
#         # and returns the distance between these nodes.
#         dist_between_nodes = CreateDistanceCallback(adjacency_matrix)
#         dist_callback = dist_between_nodes.Distance
#         routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)
#         # Solve, returns a solution if any.
#         assignment = routing.SolveWithParameters(search_parameters)
#         if assignment:
#             # Solution cost.
#             print("Total distance: " + str(assignment.ObjectiveValue()) + " miles\n")
#             # Inspect solution.
#             # Only one route here; otherwise iterate from 0 to routing.vehicles() - 1
#             route_number = 0
#             index = routing.Start(route_number) # Index of the variable for the starting node.
#             route = ''
#             while not routing.IsEnd(index):
#                 # Convert variable indices to node indices in the displayed route.
#                 route += str(city_names[routing.IndexToNode(index)]) + ' -> '
#                 index = assignment.Value(routing.NextVar(index))
#             route += str(city_names[routing.IndexToNode(index)])
#             print("Route:\n\n" + route)
#         else:
#             print('No solution found.')
#     else:
#         print('Specify an instance greater than 0.')

In [75]:
contries_to_visit = []
contries_idx = []
for i,c in enumerate(list_of_kingdom_names):
    if sc.s[i]:
        contries_to_visit.append(c)
        contries_idx.append(i)
print(contries_to_visit)
print(contries_idx)

['Hera', 'Aether', 'Chaos', 'Aphrodite', 'Gaia', 'Eros', 'Pontus', 'Hypnos', 'Hestia']
[0, 1, 3, 5, 7, 9, 10, 12, 13]


In [76]:
truncated_g = shortest_dist[contries_idx][:, contries_idx]

In [88]:
from pytsp import atsp_tsp, run, dumps_matrix

matrix_sym = atsp_tsp(truncated_g, strategy="avg")
outf = "/tmp/myroute.tsp"
with open(outf, 'w') as dest:
    dest.write(dumps_matrix(matrix_sym, name="My Route"))
tour = run(outf, start=list_of_kingdom_names.index(starting_kingdom), solver="LKH")
complete_visits = [contries_idx[i] for i in tour['tour']]
complete_visits += [complete_visits[0]]

In [89]:
from functools import reduce
p = [complete_visits[0]] + reduce(lambda x,y: x+y, [recon[complete_visits[i]][complete_visits[i+1]][1:] for i in range(1, len(complete_visits) -1)])
print(p)

[0, 2, 3, 2, 5, 8, 11, 10, 11, 13, 11, 12, 11, 8, 9, 8, 7, 8, 5, 2, 0]


In [90]:
kindoms_visited = [list_of_kingdom_names[j] for j in p]
kv = reduce(lambda x,y: x+' '+y, kindoms_visited)
print(kv)

Hera Dionysus Chaos Dionysus Aphrodite Ares Hades Pontus Hades Hestia Hades Hypnos Hades Ares Eros Ares Gaia Ares Aphrodite Dionysus Hera


In [91]:
kindoms_conquered = contries_to_visit
kc = reduce(lambda x,y: x+' '+y, kindoms_conquered)
print(kc)

Hera Aether Chaos Aphrodite Gaia Eros Pontus Hypnos Hestia


In [ ]:
f = open('0.out', 'w')
f.write(kv)
f.write(kc)

In [92]:
from SetCoverPy import *